In [80]:
import os
import sys

sys.path.append("../")

In [81]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from src.ml.sinkhorn import SinkhornLoss, sinkhorn

In [85]:
n_classes=10

In [86]:
# Load model
model = torchvision.models.alexnet(pretrained=False, num_classes=n_classes)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [26]:
class CIFAR10Instance(torchvision.datasets.CIFAR10):
    """
    https://github.com/yukimasano/self-label/blob/581957c2fcb3f14a0382cf71a3d36b21b9943798/cifar_utils.py#L5
    """
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(CIFAR10Instance, self).__init__(root=root,
                                                           train=train,
                                                           transform=transform,
                                                           target_transform=target_transform)


    def __getitem__(self, index):
        image, target = self.data[index], self.targets[index]
        image = Image.fromarray(image)

        if self.transform is not None:
            img = self.transform(image)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index

In [27]:
# Load CIFAR-10
transform_train = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomResizedCrop(size=224, scale=(0.2, 1.)),
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]
)

transform_test = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomResizedCrop(size=224, scale=(0.2, 1.)),
        transforms.ToTensor(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]
)

trainset = CIFAR10Instance(
    root="../data/cifar-10",
    train=True,
    download=True,
    transform=transform_train
)
testset = CIFAR10Instance(
    root="../data/cifar-10",
    train=False,
    download=True,
    transform=transform_test
)
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=16,
    shuffle=True,
    num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=100,
    shuffle=True,
    num_workers=2
)

classes = (
    "plane", "car", "bird", "cat",
    "deer", "dog", "frog", "horse", 
    "ship", "truck"
)

In [49]:
N = len(trainloader.dataset)
n_clusters = 128

selflabels = np.zeros(N, dtype=np.int32)

for qq in range(N):
    selflabels[qq] = qq % n_clusters
    
selflabels = np.random.permutation(selflabels)
selflabels = torch.LongTensor(selflabels)

selflabels_onehot = torch.nn.functional.one_hot(
    selflabels,
    num_classes=n_clusters
)

In [74]:
for batch_idx, (inputs, targets, indexes) in enumerate(trainloader):
    scores = model(inputs)
    log_proba = torch.nn.LogSoftmax(dim=1)(scores)
    break

In [104]:
a = torch.ones(n_classes) / n_classes
b = torch.ones(16) / N

loss = SinkhornLoss(
    a=a,
    b=b,
    solver=sinkhorn,
    n_iter=10,
    epsilon=.1
)

In [106]:
loss(log_proba, selflabels_onehot)

RuntimeError: size mismatch, [16 x 10], [16] at ../aten/src/TH/generic/THTensorMath.cpp:193